In [104]:
# -*- coding: utf-8 -*-
from Modelo import Pessoa, Destino, Funcoes
from osmnx import load_graphml, graph_from_point, save_graphml, nearest_nodes, graph_to_gdfs
from os import path, makedirs



In [105]:
funcoes = Funcoes()

# coordenada inicial - GARAGEM

coordenadaInicial = [-22.9105756,-42.8363557]
localizacaoAtual = coordenadaInicial

# PEGANDO DADOS DO JSON
lista = funcoes.lerArquivoJson("dados.txt")


Lendo Json...


In [106]:

# PREPARANDO LISTA DE ALUNOS EM OBJETOS
alunos = []

for aluno in lista['alunos']:
    alunos.append(Pessoa(aluno['id'], aluno['nome'], aluno['idEscola'], aluno['coordenadas']))


# PREPARANDO LISTA DE ESCOLAS QUE TEM ALUNOS A RECEBER EM OBJETOS
escolas = []
escolasIdCadastradas = []

for aluno in alunos:
    if aluno.idDestino not in escolasIdCadastradas:
        escolaEmCadastro = Destino(lista['escolas'][aluno.idDestino]['id'], lista['escolas'][aluno.idDestino]['nome'], lista['escolas'][aluno.idDestino]['coordenadas'])
        escolaEmCadastro.adicionarVisitante(aluno)
        escolas.append(escolaEmCadastro)
        escolasIdCadastradas.append(aluno.idDestino)
    else:
        for escola in escolas:
            if escola.id == aluno.idDestino:
                escola.adicionarVisitante(aluno)


In [107]:
#GRAFOS DE RUAS

#salvando ou abrindo grafos para nao recalcular

graph_filename = "marica_drive_graph.graphml"
graph_folder = "graphs" # Pasta para armazenar os grafos

# Cria a pasta 'graphs' se ela não existir
if not path.exists(graph_folder):
    makedirs(graph_folder)

graph_filepath = path.join(graph_folder, graph_filename)


# Baixar grafo da área em torno dos pontos
G = None
if path.exists(graph_filepath):
    print(f"Carregando grafo de ruas de '{graph_filepath}'...")
    G = load_graphml(graph_filepath)
    print("Grafo de ruas carregado com sucesso.")
else:
    print(f"Baixando grafo de ruas para a área ao redor de {localizacaoAtual}...")
    G = graph_from_point(localizacaoAtual, dist=15000, network_type="drive")
    print("Grafo baixado com sucesso.")
    print(f"Salvando grafo em '{graph_filepath}'...")
    save_graphml(G, filepath=graph_filepath)
    print("Grafo salvo com sucesso.")

#le todos os nós e organiza
grafo = funcoes.cria_grafo(G)

Carregando grafo de ruas de 'graphs\marica_drive_graph.graphml'...
Grafo de ruas carregado com sucesso.


In [108]:
#CRIANDO LISTA DE BUSCAS
coordenadaInicial = [-22.9105756,-42.8363557]
listaBusca = [] 
listaPessoasRecolhidas = []
listaAlunosEntregues = []

for aluno in alunos:
    listaBusca.append(aluno)

for escola in escolas:
    listaBusca.append(escola)

In [121]:
print(f"{'='*11} PROVan {'='*11}")
if len(listaBusca) == 0:
    print(f"A Rota chegou ao fim!")
    print(f"Obrigado por utilizar o PROVan!")

mapa, listaBusca, coordenadaInicial, listaPessoasRecolhidas = funcoes.plotarMapa(coordenadaInicial, listaBusca, grafo, G, listaPessoasRecolhidas, 0)
listaPontosJaVisitados = listaPessoasRecolhidas[:-1]
if len(listaPontosJaVisitados) > 0:
    print(f"Localização atual: {listaPontosJaVisitados[-1].nome}")
print(f"Destino atual: {listaPessoasRecolhidas[-1].nome}")
print(f"{'='*30}")
print(f"Destinos Visitados: {len(listaPontosJaVisitados)}")
acentosOcupados = 0
alunosEntregues = 0

if len(listaPontosJaVisitados) > 0 :
    for pontos in listaPontosJaVisitados:
        print(pontos.nome, end='; ')
        if isinstance(pontos, Pessoa):
            acentosOcupados += 1

        elif isinstance(listaPontosJaVisitados[-1], Destino):
            for aluno in listaPontosJaVisitados[-1].visitantes:
                alunosEntregues += 1
                acentosOcupados -= 1
                listaAlunosEntregues.append(aluno)
        
print()
print(f"{'='*30}")
print("Alunos entregues:")
for aluno in listaAlunosEntregues:
    print(aluno.nome, end='; ')

print()
print(f"{'='*30}")





mapa


=========== PROVan ===========
A Rota chegou ao fim!
Obrigado por utilizar o PROVan!


IndexError: list index out of range